In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import re
import os

In [ ]:
results_dir = 'results/'
fig_dir = os.path.join('figures/')
box_dir = os.path.join(fig_dir, 'box_dir/')
default_width = 1000
default_height = 750

if not os.path.exists(fig_dir):
    os.makedirs(fig_dir)

if not os.path.exists(box_dir):
    os.makedirs(box_dir)

In [ ]:
data = pd.read_csv(os.path.join(results_dir,'transformed_distance_query_results_root_Los_Angeles-_CA.csv'))

In [ ]:
def add_confident_column(df):
    def classify_distance(distance):
        # Convert to string to handle both float and int
        distance_str = str(distance)
        
        # Check if the distance ends with two zeros before the decimal point
        if re.search(r'00\.?0*$', distance_str):
            return 'guess'
        else:
            return 'confident'
    
    df['confidence'] = df['extracted_digits'].apply(classify_distance)
    return df
    
data = add_confident_column(data)

In [ ]:
confidence_cluster_classifier = [3461, 4563, 7804, 14813, 2453, 13496, 2558, 11464, 1615, 8690]

# filter data to be only index in confidence_cluster_2, sae_layer_21 and feature_type = end
filter_data = data[(data['index'].isin(confidence_cluster_classifier)) & (data['sae_layer'] == 21) & (data['feature_type'] == 'end')]

In [ ]:
filter_data

In [ ]:
filter_data.head()

In [ ]:
# fig.write_image(os.path.join(dist_dir, "scatter_obs_vs_expec_dist.png"), scale = 3.0)
# fig.write_image('test.png', scale = 3.0)

In [ ]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

df = filter_data.copy()

# Get unique indices
unique_indices = df['index'].unique()

# Color palette
color_palette = {'guess': 'lightblue', 'confident': 'lightcoral'}

# Create a separate plot for each index
for index in unique_indices:
    # Filter data for the current index
    index_data = df[df['index'] == index]
    
    # Create the plot
    plt.figure(figsize=(6, 6))
    
    # Create the violin plot
    sns.violinplot(data=index_data, x='confidence', y='activation', 
                   inner="box", palette=color_palette)
    
    plt.title(f'Activation Distribution for Index: {index}', fontsize=16)
    plt.xlabel('Confidence', fontsize=12)
    plt.ylabel('Activation', fontsize=12)
    
    # Perform Mann-Whitney U test
    confident = index_data[index_data['confidence'] == 'confident']['activation']
    guess = index_data[index_data['confidence'] == 'guess']['activation']
    
    if len(confident) > 0 and len(guess) > 0:
        statistic, p_value = stats.mannwhitneyu(confident, guess, alternative='two-sided')
        
        y_max = index_data['activation'].max()
        y_min = index_data['activation'].min()
        y_range = y_max - y_min
        
        plt.plot([0, 1], [y_max + y_range * 0.05] * 2, color='black')
        plt.text(0.5, y_max + y_range * 0.07, f'p={p_value:.3f}', ha='center')

    plt.tight_layout()
    plt.savefig(os.path.join(box_dir, f'feature_importance_layer_21_type_end_feature_{index}.png'), dpi = 300)
    plt.show()

# Print summary statistics
print(df.groupby(['index', 'confidence'])['activation'].describe())